<a href="https://colab.research.google.com/github/guarnieri0/GAIN/blob/master/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import librerie**

In [ ]:
pip install -U sentence-transformers

In [17]:
import pandas as pd 
import json
import datetime
from sklearn.preprocessing import LabelEncoder
import numpy as np
from numpy.linalg import norm
from numpy import savetxt
from sklearn import preprocessing
from openpyxl import Workbook
from openpyxl import load_workbook
from sentence_transformers import SentenceTransformer, util
import torch

**Reading Dataset**

In [ ]:
df = pd.read_json("dataset.json")

**Deleting useless columns**

In [ ]:
df = df.drop(['idx','testArgs','lastFailJob','engine','assignee','lastPassJob','projectIdx','validationIdx','templateIdx','triggerIdx','firstFailJob','lastFailJob','silent','status','reportLink'], axis = 1)


**Retrieving the Error Code and Error description**

In [ ]:
df_splitted= df['compareInfo'].str.split(pat=',', expand=True)
df_splitted = df_splitted.drop([0,1], axis = 1)
error_code = df_splitted.iloc[:,0].to_list()
error_description = df_splitted.iloc[:,1].to_list()
df.insert(5, 'error_description', error_description)
df.insert(6,'error_code',error_code)



**Retrieving the test category**

In [ ]:
testname_splitted = df['testName'].str.split(pat='\\', expand = True)
testname_splitted = testname_splitted.drop([0,1,2,4,5], axis = 1)
testname = testname_splitted.iloc[:,0].to_list()
df.insert(7,'cat_name', testname)



**Deleting redundant columns**

In [ ]:
df = df.drop(['compareInfo','reportTitle'], axis = 1)

**Filling the NULL value**

---

In [ ]:
df['error_description'] = df['error_description'].fillna('None')
df['error_code'] = df['error_code'].fillna('None')


**Encoding text features**

In [ ]:
encoder_df_error_code = LabelEncoder()
encoder_df_error_description = LabelEncoder()
encoder_df_cat_name = LabelEncoder()
df['error_code'] = encoder_df_error_code.fit_transform(df['error_code'])
df['error_description'] = encoder_df_error_description.fit_transform(df['error_description'])
df['cat_name'] = encoder_df_cat_name.fit_transform(df['cat_name'])



**This print gives the results of the encoding**

In [ ]:
print(df.max())

testName               test_verification
reportId               MNDORADOVAL-31553
timestamp            2022-10-10 02:26:24
error_description                     17
error_code                            21
cat_name                              75
dtype: object


**Divide in two datasets (one of "training" and one of testing), (training : before today, testing : today)**

In [ ]:
ts =  pd.Timestamp(year = 2022, month = 10, day = 9, hour =0 , second = 0)
df_past = df.loc[lambda df: df['timestamp'] < ts]
df = df.drop(df[df.timestamp < ts].index)

**Deleting time from timestamp (only date)**

In [ ]:
df['timestamp'] = df['timestamp'].dt.date
df_past['timestamp'] = df_past['timestamp'].dt.date


# **Calculate the similarity matrix**

In [ ]:
similarity = np.empty((df_past.shape[0],df.shape[0]))
for i in range (0,df_past.shape[0],1):
  for j in range (0, df.shape[0],1):  
    past_vector = np.array([df_past.iloc[i][3],df_past.iloc[i][4],df_past.iloc[i][5]])
    today_vector = np.array([df.iloc[j][3],df.iloc[j][4],df.iloc[j][5]])
    #cosin similarity
    similarity[i,j] = np.dot(past_vector,today_vector)/(norm(past_vector)*norm(today_vector))

# **Decoding the features**

In [ ]:
df['error_code'] = encoder_df_error_code.inverse_transform(df['error_code'])
df['error_description'] = encoder_df_error_description.inverse_transform(df['error_description'])
df['cat_name'] = encoder_df_cat_name.inverse_transform(df['cat_name'])

# **Saving the test set**

In [ ]:
df.to_excel('Matches.xlsx', sheet_name='Sightings Tested')

# **Reporting the results obtained into the same file**

In [ ]:
wb = load_workbook('Matches.xlsx')
ws1 = wb['Sightings Tested']
for i in range(0,similarity.shape[1],1):
  best_match_array = np.where(similarity[:,i] == np.amax(similarity[:,i]))
  best_match = best_match_array[0]
  for j in range(0,len(best_match),1):
     ws1.cell(i+2,j+8).value = df_past.iloc[best_match[j]][1]
wb.save('Matches.xlsx')

# Root cause Text Analysis

In [18]:
pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
rcdf = pd.read_excel("/content/Error_with_RootCause.xls", index_col = 1)
rcdf.reset_index(inplace = True)

In [20]:
rcdf.columns

Index(['errorcodeid', 'tfdid', 'description', 'details'], dtype='object')

In [21]:
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')

In [22]:
device = torch.device('cuda:0')
model = model.to(device)

In [23]:
print(rcdf.shape)
rcdf.dropna(how = 'any', inplace= True)
print(rcdf.shape)

(65535, 4)
(4238, 4)


In [ ]:
rcdf_past = rcdf.iloc[0:200]
print(rcdf_past)
rcdf = rcdf.iloc[200:]
print(rcdf)

In [25]:
rcdf.shape
rcdf_past.shape
simil_mat = np.empty((rcdf_past.shape[0],rcdf.shape[0]))

In [ ]:
for i in range (0,rcdf_past.shape[0],1):
  for j in range (0, rcdf.shape[0],1):
    vector_sentences = [rcdf.iloc[j][3],rcdf_past.iloc[i][3]]
    embeddings = model.encode(vector_sentences)
    cosine_scores = util.cos_sim(embeddings[0],embeddings[1])
    simil_mat[i][j] = cosine_scores[0,0].numpy()


In [34]:
print(simil_mat)

In [35]:
print(embeddings.shape)

(3, 512)


In [ ]:
cosine_scores_1_2 = util.cos_sim(embeddings[0],embeddings[1])
cosine_scores_1_3 = util.cos_sim(embeddings[0],embeddings[2])

In [38]:
print(cosine_scores_1_2)
print(cosine_scores_1_3)

tensor([[0.9873]])
tensor([[0.3400]])


In [78]:
cosine_scores_1_2.shape

torch.Size([1, 1])

In [81]:
a = cosine_scores_1_2[0,0].numpy()
print(a)

0.9872546
